# A Lot of Models
> https://www.kaggle.com/code/ysthehurricane/advanced-stock-pred-using-svr-rfr-knn-lstm-gru/notebook

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import math
import datetime as dt
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, GRU

from itertools import cycle

# ! pip install plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
import os
for dirname, _, filenames in os.walk('D:/kaggle/jpx-prediction'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

D:/kaggle/jpx-prediction\stock_list.csv
D:/kaggle/jpx-prediction\data_specifications\options_spec.csv
D:/kaggle/jpx-prediction\data_specifications\stock_fin_spec.csv
D:/kaggle/jpx-prediction\data_specifications\stock_list_spec.csv
D:/kaggle/jpx-prediction\data_specifications\stock_price_spec.csv
D:/kaggle/jpx-prediction\data_specifications\trades_spec.csv
D:/kaggle/jpx-prediction\example_test_files\financials.csv
D:/kaggle/jpx-prediction\example_test_files\options.csv
D:/kaggle/jpx-prediction\example_test_files\sample_submission.csv
D:/kaggle/jpx-prediction\example_test_files\secondary_stock_prices.csv
D:/kaggle/jpx-prediction\example_test_files\stock_prices.csv
D:/kaggle/jpx-prediction\example_test_files\trades.csv
D:/kaggle/jpx-prediction\jpx_tokyo_market_prediction\__init__.py
D:/kaggle/jpx-prediction\jpx_tokyo_market_prediction\competition.cpython-37m-x86_64-linux-gnu.so
D:/kaggle/jpx-prediction\supplemental_files\financials.csv
D:/kaggle/jpx-prediction\supplemental_files\options.c

## Load Dataset

In [3]:
df=pd.read_csv('D:/kaggle/jpx-prediction/train_files/stock_prices.csv')
df

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026
...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,20211203_9990,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,1.0,NaN,False,0.034816
2332527,20211203_9991,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,1.0,NaN,False,0.025478
2332528,20211203_9993,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1.0,NaN,False,-0.004302
2332529,20211203_9994,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,1.0,NaN,False,0.009098


### Drop the RowId

In [4]:
df.drop('RowId', axis=1, inplace=True)
df.head()

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026


### Change String Date to Datetime

In [5]:
df["Date"] = pd.to_datetime(df['Date'])

## Choose Five Securities

In [10]:
sec_1301 = df[df["SecuritiesCode"]==1301].reset_index(drop=True)
sec_1301.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1202 entries, 0 to 1201
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              1202 non-null   datetime64[ns]
 1   SecuritiesCode    1202 non-null   int64         
 2   Open              1201 non-null   float64       
 3   High              1201 non-null   float64       
 4   Low               1201 non-null   float64       
 5   Close             1201 non-null   float64       
 6   Volume            1202 non-null   int64         
 7   AdjustmentFactor  1202 non-null   float64       
 8   ExpectedDividend  5 non-null      float64       
 9   SupervisionFlag   1202 non-null   bool          
 10  Target            1202 non-null   float64       
dtypes: bool(1), datetime64[ns](1), float64(7), int64(2)
memory usage: 95.2 KB


In [11]:
sec_4768 = df[df["SecuritiesCode"]==4768].reset_index(drop=True)
sec_4768.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1202 entries, 0 to 1201
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              1202 non-null   datetime64[ns]
 1   SecuritiesCode    1202 non-null   int64         
 2   Open              1201 non-null   float64       
 3   High              1201 non-null   float64       
 4   Low               1201 non-null   float64       
 5   Close             1201 non-null   float64       
 6   Volume            1202 non-null   int64         
 7   AdjustmentFactor  1202 non-null   float64       
 8   ExpectedDividend  9 non-null      float64       
 9   SupervisionFlag   1202 non-null   bool          
 10  Target            1202 non-null   float64       
dtypes: bool(1), datetime64[ns](1), float64(7), int64(2)
memory usage: 95.2 KB


In [12]:
sec_6474 = df[df["SecuritiesCode"]==6474].reset_index(drop=True)
sec_6474.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1202 entries, 0 to 1201
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              1202 non-null   datetime64[ns]
 1   SecuritiesCode    1202 non-null   int64         
 2   Open              1201 non-null   float64       
 3   High              1201 non-null   float64       
 4   Low               1201 non-null   float64       
 5   Close             1201 non-null   float64       
 6   Volume            1202 non-null   int64         
 7   AdjustmentFactor  1202 non-null   float64       
 8   ExpectedDividend  10 non-null     float64       
 9   SupervisionFlag   1202 non-null   bool          
 10  Target            1202 non-null   float64       
dtypes: bool(1), datetime64[ns](1), float64(7), int64(2)
memory usage: 95.2 KB


In [13]:
sec_1323 = df[df["SecuritiesCode"]==1323].reset_index(drop=True)
sec_1323.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              0 non-null      datetime64[ns]
 1   SecuritiesCode    0 non-null      int64         
 2   Open              0 non-null      float64       
 3   High              0 non-null      float64       
 4   Low               0 non-null      float64       
 5   Close             0 non-null      float64       
 6   Volume            0 non-null      int64         
 7   AdjustmentFactor  0 non-null      float64       
 8   ExpectedDividend  0 non-null      float64       
 9   SupervisionFlag   0 non-null      bool          
 10  Target            0 non-null      float64       
dtypes: bool(1), datetime64[ns](1), float64(7), int64(2)
memory usage: 124.0 bytes


In [14]:
sec_2288 = df[df["SecuritiesCode"]==2288].reset_index(drop=True)
sec_2288.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1202 entries, 0 to 1201
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              1202 non-null   datetime64[ns]
 1   SecuritiesCode    1202 non-null   int64         
 2   Open              1201 non-null   float64       
 3   High              1201 non-null   float64       
 4   Low               1201 non-null   float64       
 5   Close             1201 non-null   float64       
 6   Volume            1202 non-null   int64         
 7   AdjustmentFactor  1202 non-null   float64       
 8   ExpectedDividend  5 non-null      float64       
 9   SupervisionFlag   1202 non-null   bool          
 10  Target            1202 non-null   float64       
dtypes: bool(1), datetime64[ns](1), float64(7), int64(2)
memory usage: 95.2 KB


## Check for Null Values and Adjust

In [15]:
sec_1301.isnull().sum()

Date                   0
SecuritiesCode         0
Open                   1
High                   1
Low                    1
Close                  1
Volume                 0
AdjustmentFactor       0
ExpectedDividend    1197
SupervisionFlag        0
Target                 0
dtype: int64

In [19]:
sec_1301["Open"] = sec_1301["Open"].interpolate()
sec_1301["High"] = sec_1301["High"].interpolate()
sec_1301["Low"] = sec_1301["Low"].interpolate()
sec_1301["Close"] = sec_1301["Close"].interpolate()
sec_1301.isnull().sum()

Date                   0
SecuritiesCode         0
Open                   0
High                   0
Low                    0
Close                  0
Volume                 0
AdjustmentFactor       0
ExpectedDividend    1197
SupervisionFlag        0
Target                 0
dtype: int64

In [22]:
sec_1301 = sec_1301.fillna(0)
sec_1301.isnull().sum()

Date                0
SecuritiesCode      0
Open                0
High                0
Low                 0
Close               0
Volume              0
AdjustmentFactor    0
ExpectedDividend    0
SupervisionFlag     0
Target              0
dtype: int64

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
911,2020-09-29,1301,2800.0,2857.0,2790.0,2846.0,38100,1.0,0.0,False,0.000000
912,2020-09-30,1301,2828.0,2849.0,2798.0,2798.0,21900,1.0,0.0,False,-0.021086
913,2020-10-01,1301,2813.5,2829.5,2768.0,2768.5,0,1.0,0.0,False,0.029208
914,2020-10-02,1301,2799.0,2810.0,2738.0,2739.0,19000,1.0,0.0,False,0.001774
